In [3]:
#compile this cell first
import time
import numpy as np
import matplotlib.pyplot as plt
%matplotlib auto
from mpl_toolkits.mplot3d.axes3d import Axes3D
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.cm as cm
from random import randint

#PARAMETERS
fire_decay = 0.05
evaporation_rate = 0.3
rain_threshold = 34
rain_rate = 0.4
vege_drain = 0.03
vege_transpiration = 0.06
water_dry = 11.5 #dryness threshold
dry_threshold = 5 #number of iterations for the cell to become dry
max_time = 101 #number of iterations
animation_speed = 0.001 #delay beteen frames, in seconds


SLOPE = 1
def c_sigma(x):
    return 40/(1+np.exp(-SLOPE*(x-(40-30)/2)))
def v_sigma(x):
    return 30/(1+np.exp(-SLOPE*(x-(30-20)/2)))
def w_sigma(x):
    return 20/(1+np.exp(-SLOPE*(x-(20-10)/2)))
def f_sigma(x):
    return 10/(1+np.exp(-SLOPE*(x-(10-0)/2)))

class System:
    counter = 0
    def __init__(self, N, M, ax, fig):
        self.N = N
        self.M = M
        self.ax = ax
        self.fig = fig
        #INITIAL PARAMETERS
        self.clouds = np.random.normal(38, 1, size=(N,M)) #30-40
        self.vege = np.random.normal(24, 4, size=(N,M))   #20-30
        self.water = np.random.normal(12, 1, size=(N,M))  #10-20
        self.fire = np.random.normal(0.00001, 0.00000001, size=(N,M)) #0-10
        self.dry = np.zeros((N,M))
        self.counter = 0
        
    def draw(self):
        self.ax.cla()
        X = np.arange(0, 10, 1)
        Y = np.arange(0, 10, 1)
        X, Y = np.meshgrid(X, Y)
        self.ax.plot_surface(X, Y, self.clouds, rstride=1, cstride=1, linewidth=0, cmap=cm.get_cmap('Greys', 256), alpha=0.5)
        self.ax.plot_surface(X, Y, self.vege, rstride=1, cstride=1, linewidth=0, cmap=cm.get_cmap('Greens', 256), alpha=0.5)
        self.ax.plot_surface(X, Y, self.water, rstride=1, cstride=1, linewidth=0, cmap=cm.get_cmap('Blues', 256), alpha=0.5)
        self.ax.plot_surface(X, Y, self.fire, rstride=1, cstride=1, linewidth=0, cmap=cm.get_cmap('Reds', 256), alpha=0.5)
        
        name = "snapshot" + str(self.counter) + ".png"
        self.fig.savefig(name)    
    
    def neighbors(self,i, j):
        outlist = []
        for a in [i-1,i,i+1]:
            for b in [j-1, j, j+1]:
                if a in range(self.N) and b in range(self.M) and not (a==i and b==j):
                    outlist.append((a,b))
        return outlist
    
    def evaporation(self, i,j): 
        evaporated = (self.water[i,j]-10)*evaporation_rate
        transpired = (self.vege[i,j] - 20) * vege_transpiration
        neigh = self.neighbors(i,j)
        self.water[i,j] -= evaporated
        self.clouds[i,j] +=(evaporated + transpired)/2
        transpired /=2
        evaporated/=2
        for cell in neigh:
            i = cell[0]
            j = cell[1]
            self.clouds[i,j] += (evaporated+transpired)/len(neigh)
        return
    
    def rain(self, i, j):
        rained = (self.clouds[i,j]-30)*rain_rate
        neigh = self.neighbors(i,j)
        self.clouds[i,j] -= rained
        if self.water[i,j] + rained/2 <20:
            self.water[i,j] += rained/2
        else:
            self.water[i,j] = 20
        rained /= 2
        for cell in neigh:
            i = cell[0]
            j = cell[1]
            if self.water[i,j] + rained/len(neigh) <20:
                self.water[i,j] += rained/len(neigh)
            else:
                self.water[i,j] = 20
            
    def growth(self, i, j):
        if self.vege[i,j] >=29.7:
            return
        water_drain = 0 #running sum
        neigh = self.neighbors(i,j)
        for cell in neigh:
            k = cell[0]
            l = cell[1]
            if self.water[k,l] > water_dry:
                drain = self.vege[k,l]*vege_drain/2
                self.water[k,l] -= drain
                water_drain += drain
        if self.water[i,j] > water_dry:
            drain = self.water[i,j]*vege_drain
            water_drain += drain
            self.water[i,j] -= drain
        if self.vege[i,j] + water_drain >30:
            self.vege[i,j] = self.vege[i,j] = 30
        else:
            self.vege[i,j] += water_drain
      
    def dying(self, i, j):
        if self.water[i,j] <= water_dry and self.vege[i,j] >= 20.5:
            self.vege[i,j] -= (self.vege[i,j]-20)*0.1
        if self.fire[i,j]>0.5 and self.vege[i,j] >21:
            self.vege[i,j] -= self.fire[i,j]
        #if self.vege[i,j] >=30:
            #self.vege[i,j] = 30
        if self.vege[i,j] <= 20:
            self.vege[i,j] = 20
            
    def lightning(self):
        i = randint(0,9)
        j = randint(0,9)
        if randint(0,100) >85 and self.dry[i,j] >= dry_threshold:
            self.fire[i,j] += 8
            if self.fire[i,j] >=10:
                self.fire[i,j] = 10
                self.vege[i,j] = 20
        
    def step(self):
        running_sum_c = 0
        running_sum_v = 0
        running_sum_w = 0
        running_sum_f = 0
        for x in range(self.N):
            for y in range(self.M):
                if(self.water[x,y] > 11):
                    self.evaporation(x,y)
                    
                if(self.clouds[x,y] > rain_threshold):
                    self.rain(x,y) 
                self.growth(x,y) 
                self.dying(x,y)
                if self.water[x,y] < water_dry:
                    self.dry[x,y] +=1
                    if self.dry[x,y] > dry_threshold:
                        self.dry[x,y] = dry_threshold
                else:
                    self.dry[x,y] = 0
                if self.fire[x,y] >3:
                    neigh = self.neighbors(x,y)
                    for cell in neigh:
                        k = cell[0]
                        l = cell[1]
                        if self.dry[k,l] > dry_threshold:
                            self.fire[k,l] += self.fire[x,y]*0.8
                self.fire[x,y] -= self.fire[x,y]*fire_decay 
                    
            
        self.lightning()
        self.counter +=1
    '''    
        if self.counter >100:
            if randint(0,100) < 10:
                #gejzer
                self.water[randint(0,9), randint(0,9)] = 20
        self.counter+=1
        pass
    '''
    
    def sum_activities(self):
        c = 0
        v = 0
        w = 0
        f = 0
        for x in range(self.N):
            for y in range(self.M):
                c += self.clouds[x,y] - 30
                v += self.vege[x,y] - 20
                w += self.water[x,y] - 10
                f += self.fire[x,y]
        return c, v, w, f
                
    
    def run(self, iterations, delay):
        self.draw()
        density = 11
        a = 5
        b = 5
        x = np.linspace(0, iterations-1, density)
        c = np.zeros(density)
        v = np.zeros(density)
        w = np.zeros(density)
        f = np.zeros(density)
        C = np.zeros(density)
        V = np.zeros(density)
        W = np.zeros(density)
        F = np.zeros(density)
        '''print(x)
        print(c)
        print(v)
        print(w)
        print(f)'''
        c[0] = self.clouds[a,b]
        v[0] = self.vege[a,b]
        w[0] = self.water[a,b]
        f[0] = self.fire[a,b]
        C[0], V[0], W[0], F[0] = self.sum_activities()
        
        for i in range(iterations):
            self.step()
            if i%10 ==0:
                c[i/10] = self.clouds[a,b]
                v[i/10] = self.vege[a,b]
                w[i/10] = self.water[a,b]
                f[i/10] = self.fire[a,b]
                C[i/10], V[i/10], W[i/10], F[i/10] = self.sum_activities()
            
            self.draw()
            plt.pause(delay)
        '''print(x)
        print(C)
        print(V)
        print(W)
        print(F)'''
        fig = plt.figure()

        axes = fig.add_axes([0.1, 0.1, 0.8, 0.8]) # left, bottom, width, height (range 0 to 1)

        axes.plot(x, c, 'grey')
        axes.plot(x, v, 'g')
        axes.plot(x, w, 'b')
        axes.plot(x, f, 'r')

        axes.set_xlabel('iteracja')
        axes.set_ylabel('poziom')
        axes.set_title('poziom w komorce 5,5')
        fig.savefig("single_cell.png")
        fig2 = plt.figure()
        axes2 = fig2.add_axes([0.1, 0.1, 0.8, 0.8]) # left, bottom, width, height (range 0 to 1)
        axes2.plot(x, C, 'grey')
        axes2.plot(x, V, 'g')
        axes2.plot(x, W, 'b')
        axes2.plot(x, F, 'r')

        axes2.set_xlabel('iteracja')
        axes2.set_ylabel('poziom')
        axes2.set_title('suma aktywnosci w systemie')
        fig2.savefig("all_activities.png")

Using matplotlib backend: TkAgg


In [5]:
fig = plt.figure(figsize=(10,10))
ax = fig.gca(projection='3d')
sys = System(10,10,ax, fig)
sys.run(max_time, animation_speed)